In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

In [3]:
train = pd.read_csv("Train.csv")
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [5]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50
count,53471.000000,52156.000000,55645.000000,55645.000000,55645.000000,4.264000e+04,5.083700e+04,4.264000e+04,4.799200e+04,5.564500e+04,...,12271.000000,17771.000000,54677.000000,55432.000000,50617.000000,24683.000000,40096.000000,3.511100e+04,55645.000000,4.494400e+04
mean,3491.795665,0.550737,1.052225,0.851979,1.956317,6.244479e+05,6.865210e+06,2.626690e+06,1.316002e+07,1.185585e+07,...,147.797977,108.117363,0.368215,6.634511,0.563377,0.072520,0.095371,3.052449e+05,1.049061,6.005862e+05
std,188.462426,0.820979,2.147768,3.157692,10.512396,1.433422e+06,1.912729e+07,3.927355e+06,1.977963e+07,2.669459e+07,...,45.085889,36.765769,0.412858,6.378946,0.196973,0.291146,0.335210,1.647757e+06,1.806160,5.842405e+06
min,2990.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3358.000000,0.070788,0.000000,0.000000,0.000000,1.400400e+04,6.869740e+05,1.929440e+05,1.368502e+06,4.500730e+05,...,136.191750,87.055500,0.000000,2.020000,0.411672,0.000000,0.000000,7.710079e+01,0.000000,4.560141e-02
50%,3484.000000,0.267575,0.062000,0.000000,0.000000,1.155330e+05,2.704328e+06,9.639420e+05,5.506295e+06,3.707559e+06,...,150.000000,118.441500,0.220000,5.050000,0.588648,0.000000,0.000000,3.181243e+02,0.000000,1.703996e-01
75%,3620.000000,0.719512,1.282000,0.000000,0.000000,5.259280e+05,6.993831e+06,3.751516e+06,1.694552e+07,1.326101e+07,...,167.754000,137.496750,0.628573,10.100000,0.753840,0.000000,0.000000,1.153022e+03,1.181754,4.993370e-01
max,3900.000000,18.015050,57.371600,91.672200,407.748600,5.313546e+07,2.158794e+09,1.037397e+08,3.200533e+08,2.191361e+09,...,645.450000,255.000000,2.200000,91.910000,0.800000,5.000000,5.000000,5.321014e+07,28.000000,2.307571e+08


In [6]:
train.isnull().sum()/len(train) * 100

Applicant_ID       0.000000
form_field1        4.516071
form_field2        6.864286
form_field3        0.633929
form_field4        0.633929
form_field5        0.633929
form_field6       23.857143
form_field7        9.219643
form_field8       23.857143
form_field9       14.300000
form_field10       0.633929
form_field11      56.108929
form_field12      17.669643
form_field13      10.516071
form_field14       0.000000
form_field15      40.133929
form_field16      23.278571
form_field17      19.912500
form_field18      18.575000
form_field19       0.007143
form_field20       0.633929
form_field21      28.310714
form_field22      36.428571
form_field23      50.219643
form_field24      23.744643
form_field25       9.732143
form_field26      13.282143
form_field27      16.605357
form_field28       0.633929
form_field29       0.633929
form_field30      45.551786
form_field31      70.371429
form_field32       9.732143
form_field33       2.242857
form_field34       0.633929
form_field35      41

In [8]:
new_train = train.drop(columns=['form_field31','form_field40','form_field41'])

In [9]:
test.isnull().sum()/len(test) * 100

Applicant_ID     0.000000
form_field1      4.625000
form_field2      7.120833
form_field3      0.608333
form_field4      0.608333
form_field5      0.608333
form_field6     23.350000
form_field7      9.295833
form_field8     23.350000
form_field9     14.166667
form_field10     0.612500
form_field11    55.825000
form_field12    17.429167
form_field13    10.262500
form_field14     0.000000
form_field15    39.966667
form_field16    22.808333
form_field17    19.562500
form_field18    18.204167
form_field19     0.000000
form_field20     0.612500
form_field21    27.945833
form_field22    36.350000
form_field23    50.520833
form_field24    23.354167
form_field25     9.400000
form_field26    13.216667
form_field27    16.291667
form_field28     0.612500
form_field29     0.612500
form_field30    45.450000
form_field31    70.041667
form_field32     9.400000
form_field33     2.062500
form_field34     0.612500
form_field35    41.108333
form_field36     3.762500
form_field37     9.400000
form_field38

In [11]:
new_test = test.drop(columns=['form_field31','form_field40','form_field41'])

In [12]:
new_train.fillna(new_train.median(), inplace=True)

In [13]:
new_test.fillna(new_test.median(), inplace=True)

In [14]:
X = new_train.drop(['default_status','Applicant_ID'], axis=1)
y =new_train.default_status

In [15]:
y_ = pd.get_dummies(y)

In [16]:
X.form_field47.value_counts()

charge     36373
lending    19627
Name: form_field47, dtype: int64

In [17]:
X_ = pd.get_dummies(X)
test_ = pd.get_dummies(new_test.drop('Applicant_ID', axis=1))

In [18]:
test_.shape

(24000, 48)

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
minmax = MinMaxScaler()

In [21]:
X_scaled = minmax.fit_transform(X_)
test_sc = minmax.fit_transform(test_)

In [23]:
pip install catboost

     |████████████████████████████████| 66.0 MB 81 kB/s s eta 0:00:01   |██▏                             | 4.5 MB 3.2 MB/s eta 0:00:20     |██▌                             | 5.1 MB 3.2 MB/s eta 0:00:20     |████▋                           | 9.5 MB 3.2 MB/s eta 0:00:18MB/s eta 0:00:18     |██████                          | 12.5 MB 3.2 MB/s eta 0:00:17     |████████▎                       | 17.1 MB 39.7 MB/s eta 0:00:02     |████████████████████            | 41.2 MB 39.7 MB/s eta 0:00:01     |██████████████████████████████  | 61.8 MB 47.7 MB/s eta 0:00:01     |████████████████████████████████| 66.0 MB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 32.8 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=2d3cf7a56aec14a274847de6282b1c40e38ffb7edc12e2848f206f30855b63f0
  Stored in directory: /home/jovyan/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
You s

In [25]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
#from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

In [27]:
#from lightgbm import LGBMClassifier

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

In [29]:
cat = CatBoostClassifier()
#xgb = XGBClassifier()
#lgb = LGBMClassifier()
ada = AdaBoostClassifier()
rf = RandomForestClassifier()
lr = LogisticRegression()

In [34]:
cat.fit(X_train, y_train, eval_set=(X_test, y_test))

6:	learn: 0.4983925	test: 0.4985198	best: 0.4985198 (6)	total: 1.51s	remaining: 3m 34s
7:	learn: 0.4869662	test: 0.4873416	best: 0.4873416 (7)	total: 1.61s	remaining: 3m 20s
8:	learn: 0.4793503	test: 0.4796743	best: 0.4796743 (8)	total: 1.72s	remaining: 3m 8s
9:	learn: 0.4703752	test: 0.4706420	best: 0.4706420 (9)	total: 1.88s	remaining: 3m 5s
10:	learn: 0.4634903	test: 0.4637219	best: 0.4637219 (10)	total: 2.05s	remaining: 3m 3s
11:	learn: 0.4579992	test: 0.4584420	best: 0.4584420 (11)	total: 2.21s	remaining: 3m 2s
12:	learn: 0.4534762	test: 0.4538332	best: 0.4538332 (12)	total: 2.41s	remaining: 3m 3s
13:	learn: 0.4485022	test: 0.4486934	best: 0.4486934 (13)	total: 2.5s	remaining: 2m 56s
14:	learn: 0.4445089	test: 0.4448554	best: 0.4448554 (14)	total: 2.74s	remaining: 2m 59s
15:	learn: 0.4410589	test: 0.4413610	best: 0.4413610 (15)	total: 3.06s	remaining: 3m 8s
16:	learn: 0.4381399	test: 0.4383462	best: 0.4383462 (16)	total: 3.26s	remaining: 3m 8s
17:	learn: 0.4354796	test: 0.4357957	

In [31]:
from sklearn.metrics import roc_auc_score, log_loss

In [35]:
pred = cat.predict_proba(X_test)[:,1]

In [36]:
roc_auc_score(y_test, pred)

0.8438537757714165

In [58]:
models = [cat, xgb, lgb, ada, rf, lr]

for model in models:
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test,pred)
    print(f'the auc score for {model} is {score}')

Learning rate set to 0.052243
0:	learn: 0.6612610	total: 81ms	remaining: 1m 20s
1:	learn: 0.6326161	total: 144ms	remaining: 1m 11s
2:	learn: 0.6075513	total: 231ms	remaining: 1m 16s
3:	learn: 0.5858996	total: 314ms	remaining: 1m 18s
4:	learn: 0.5656573	total: 390ms	remaining: 1m 17s
5:	learn: 0.5512728	total: 481ms	remaining: 1m 19s
6:	learn: 0.5376713	total: 567ms	remaining: 1m 20s
7:	learn: 0.5252847	total: 639ms	remaining: 1m 19s
8:	learn: 0.5157513	total: 727ms	remaining: 1m 20s
9:	learn: 0.5056516	total: 821ms	remaining: 1m 21s
10:	learn: 0.4973231	total: 904ms	remaining: 1m 21s
11:	learn: 0.4901760	total: 995ms	remaining: 1m 21s
12:	learn: 0.4841271	total: 1.08s	remaining: 1m 21s
13:	learn: 0.4776950	total: 1.17s	remaining: 1m 22s
14:	learn: 0.4718664	total: 1.26s	remaining: 1m 23s
15:	learn: 0.4671083	total: 1.35s	remaining: 1m 22s
16:	learn: 0.4628994	total: 1.43s	remaining: 1m 22s
17:	learn: 0.4592604	total: 1.52s	remaining: 1m 22s
18:	learn: 0.4555240	total: 1.6s	remaining: 1

157:	learn: 0.3982868	total: 13s	remaining: 1m 9s
158:	learn: 0.3982270	total: 13s	remaining: 1m 8s
159:	learn: 0.3981444	total: 13.1s	remaining: 1m 8s
160:	learn: 0.3980961	total: 13.3s	remaining: 1m 9s
161:	learn: 0.3979804	total: 13.4s	remaining: 1m 9s
162:	learn: 0.3978957	total: 13.5s	remaining: 1m 9s
163:	learn: 0.3977981	total: 13.6s	remaining: 1m 9s
164:	learn: 0.3976612	total: 13.6s	remaining: 1m 9s
165:	learn: 0.3975997	total: 13.7s	remaining: 1m 8s
166:	learn: 0.3974943	total: 13.8s	remaining: 1m 8s
167:	learn: 0.3973885	total: 13.9s	remaining: 1m 8s
168:	learn: 0.3972829	total: 13.9s	remaining: 1m 8s
169:	learn: 0.3971898	total: 14s	remaining: 1m 8s
170:	learn: 0.3970765	total: 14.1s	remaining: 1m 8s
171:	learn: 0.3969383	total: 14.2s	remaining: 1m 8s
172:	learn: 0.3967980	total: 14.3s	remaining: 1m 8s
173:	learn: 0.3966981	total: 14.3s	remaining: 1m 8s
174:	learn: 0.3965721	total: 14.4s	remaining: 1m 7s
175:	learn: 0.3964919	total: 14.5s	remaining: 1m 7s
176:	learn: 0.3963

317:	learn: 0.3816776	total: 26.9s	remaining: 57.7s
318:	learn: 0.3815535	total: 27s	remaining: 57.6s
319:	learn: 0.3814971	total: 27.2s	remaining: 57.7s
320:	learn: 0.3813805	total: 27.3s	remaining: 57.7s
321:	learn: 0.3812853	total: 27.3s	remaining: 57.6s
322:	learn: 0.3811787	total: 27.4s	remaining: 57.5s
323:	learn: 0.3810857	total: 27.5s	remaining: 57.5s
324:	learn: 0.3809904	total: 27.6s	remaining: 57.4s
325:	learn: 0.3808663	total: 27.7s	remaining: 57.3s
326:	learn: 0.3807457	total: 27.8s	remaining: 57.3s
327:	learn: 0.3806376	total: 27.9s	remaining: 57.2s
328:	learn: 0.3805499	total: 28s	remaining: 57.1s
329:	learn: 0.3804199	total: 28.1s	remaining: 57s
330:	learn: 0.3803657	total: 28.2s	remaining: 56.9s
331:	learn: 0.3802391	total: 28.2s	remaining: 56.8s
332:	learn: 0.3801254	total: 28.3s	remaining: 56.7s
333:	learn: 0.3800665	total: 28.4s	remaining: 56.6s
334:	learn: 0.3799573	total: 28.5s	remaining: 56.6s
335:	learn: 0.3798370	total: 28.6s	remaining: 56.5s
336:	learn: 0.3797

476:	learn: 0.3667645	total: 39.2s	remaining: 43s
477:	learn: 0.3666837	total: 39.3s	remaining: 42.9s
478:	learn: 0.3665803	total: 39.3s	remaining: 42.8s
479:	learn: 0.3664697	total: 39.4s	remaining: 42.7s
480:	learn: 0.3663878	total: 39.5s	remaining: 42.6s
481:	learn: 0.3662960	total: 39.5s	remaining: 42.5s
482:	learn: 0.3662008	total: 39.6s	remaining: 42.4s
483:	learn: 0.3660928	total: 39.7s	remaining: 42.3s
484:	learn: 0.3659933	total: 39.8s	remaining: 42.2s
485:	learn: 0.3659176	total: 39.8s	remaining: 42.1s
486:	learn: 0.3657863	total: 39.9s	remaining: 42s
487:	learn: 0.3656937	total: 40s	remaining: 42s
488:	learn: 0.3655834	total: 40.1s	remaining: 41.9s
489:	learn: 0.3654370	total: 40.1s	remaining: 41.8s
490:	learn: 0.3653199	total: 40.2s	remaining: 41.7s
491:	learn: 0.3652472	total: 40.3s	remaining: 41.6s
492:	learn: 0.3651557	total: 40.3s	remaining: 41.5s
493:	learn: 0.3650499	total: 40.4s	remaining: 41.4s
494:	learn: 0.3649542	total: 40.5s	remaining: 41.3s
495:	learn: 0.364885

636:	learn: 0.3526738	total: 54.1s	remaining: 30.8s
637:	learn: 0.3525871	total: 54.2s	remaining: 30.7s
638:	learn: 0.3525133	total: 54.3s	remaining: 30.6s
639:	learn: 0.3524291	total: 54.3s	remaining: 30.6s
640:	learn: 0.3523394	total: 54.4s	remaining: 30.5s
641:	learn: 0.3522592	total: 54.5s	remaining: 30.4s
642:	learn: 0.3521815	total: 54.5s	remaining: 30.3s
643:	learn: 0.3520772	total: 54.6s	remaining: 30.2s
644:	learn: 0.3519708	total: 54.7s	remaining: 30.1s
645:	learn: 0.3518958	total: 54.8s	remaining: 30s
646:	learn: 0.3518096	total: 54.9s	remaining: 29.9s
647:	learn: 0.3517220	total: 54.9s	remaining: 29.8s
648:	learn: 0.3516142	total: 55s	remaining: 29.8s
649:	learn: 0.3515221	total: 55.1s	remaining: 29.7s
650:	learn: 0.3514486	total: 55.2s	remaining: 29.6s
651:	learn: 0.3513905	total: 55.2s	remaining: 29.5s
652:	learn: 0.3513040	total: 55.3s	remaining: 29.4s
653:	learn: 0.3511902	total: 55.4s	remaining: 29.3s
654:	learn: 0.3510835	total: 55.5s	remaining: 29.2s
655:	learn: 0.35

798:	learn: 0.3394223	total: 1m 6s	remaining: 16.7s
799:	learn: 0.3393581	total: 1m 6s	remaining: 16.6s
800:	learn: 0.3392615	total: 1m 6s	remaining: 16.6s
801:	learn: 0.3391549	total: 1m 6s	remaining: 16.5s
802:	learn: 0.3390827	total: 1m 6s	remaining: 16.4s
803:	learn: 0.3390134	total: 1m 6s	remaining: 16.3s
804:	learn: 0.3389505	total: 1m 6s	remaining: 16.2s
805:	learn: 0.3388653	total: 1m 7s	remaining: 16.1s
806:	learn: 0.3387694	total: 1m 7s	remaining: 16s
807:	learn: 0.3386949	total: 1m 7s	remaining: 16s
808:	learn: 0.3386295	total: 1m 7s	remaining: 15.9s
809:	learn: 0.3385458	total: 1m 7s	remaining: 15.8s
810:	learn: 0.3384685	total: 1m 7s	remaining: 15.7s
811:	learn: 0.3383981	total: 1m 7s	remaining: 15.6s
812:	learn: 0.3383314	total: 1m 7s	remaining: 15.5s
813:	learn: 0.3382731	total: 1m 7s	remaining: 15.5s
814:	learn: 0.3381973	total: 1m 7s	remaining: 15.4s
815:	learn: 0.3381087	total: 1m 7s	remaining: 15.3s
816:	learn: 0.3380413	total: 1m 7s	remaining: 15.2s
817:	learn: 0.33

954:	learn: 0.3277845	total: 1m 19s	remaining: 3.76s
955:	learn: 0.3277004	total: 1m 19s	remaining: 3.68s
956:	learn: 0.3276202	total: 1m 19s	remaining: 3.59s
957:	learn: 0.3275608	total: 1m 20s	remaining: 3.51s
958:	learn: 0.3274734	total: 1m 20s	remaining: 3.43s
959:	learn: 0.3273895	total: 1m 20s	remaining: 3.35s
960:	learn: 0.3272852	total: 1m 20s	remaining: 3.27s
961:	learn: 0.3271971	total: 1m 20s	remaining: 3.18s
962:	learn: 0.3271225	total: 1m 20s	remaining: 3.1s
963:	learn: 0.3270668	total: 1m 20s	remaining: 3.02s
964:	learn: 0.3270110	total: 1m 20s	remaining: 2.94s
965:	learn: 0.3269463	total: 1m 21s	remaining: 2.85s
966:	learn: 0.3268855	total: 1m 21s	remaining: 2.77s
967:	learn: 0.3268142	total: 1m 21s	remaining: 2.69s
968:	learn: 0.3267394	total: 1m 21s	remaining: 2.6s
969:	learn: 0.3266417	total: 1m 21s	remaining: 2.52s
970:	learn: 0.3265668	total: 1m 21s	remaining: 2.44s
971:	learn: 0.3264980	total: 1m 21s	remaining: 2.35s
972:	learn: 0.3264152	total: 1m 21s	remaining: 2

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [37]:
prediction = cat.predict_proba(test_sc)[:,1]

In [38]:
submission.head()

,Applicant_ID,default_status
0,Apcnt_1000032,1
1,Apcnt_1000048,1
2,Apcnt_1000052,1
3,Apcnt_1000076,1
4,Apcnt_1000080,1


In [39]:
submission['default_status']=prediction

In [40]:
submission.to_csv('submission2.csv', index=False)